In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import poisson

In [2]:
# list of the teams (18/19)
# data from three different sources use different names for teams. 
# manually making a dictionary to rename teams
team_name = {
            'Arsenal FC': 'Arsenal',
            'Burnley FC': 'Burnley',
            'Liverpool FC': 'Liverpool',
            'Everton FC': 'Everton',
            'Fulham FC': 'Fulham',
            'Southampton FC': 'Southampton',
            'Chelsea FC': 'Chelsea',
            'West Bromwich Albion': 'West Bromwich',
            'Manchester United': 'Manchester Utd',
            'Man United': 'Manchester Utd',
            'Man City': 'Manchester City', 
            'Newcastle': 'Newcastle Utd',
            'Newcastle United': 'Newcastle Utd',
            'West Ham United': 'West Ham Utd', 
            'Tottenham Hotspur': 'Tottenham',
            'Queens Park Rangers': 'QP Rangers',
            'Watford FC': 'Watford',
            'AFC Bournemouth': 'Bournemouth',
            'Sunderland AFC': 'Sunderland',
            'Middlesbrough FC': 'Middlesbrough',
            'Brighton & Hove Albion': 'Brighton & Hove',
            'Brighton': 'Brighton & Hove',
            'Huddersfield Town': 'Huddersfield',
            'QPR': 'QP Rangers',
            'Cardiff': 'Cardiff City',
            'Hull': 'Hull City',
            'Norwich': 'Norwich City',
            'Stoke': 'Stoke City',
            'Swansea': 'Swansea City',
            'West Brom': 'West Bromwich',
            'West Ham': 'West Ham Utd',
            'Reading FC': 'Reading',
            'Leicester': 'Leicester City'
            }

In [35]:
table_13 = pd.read_csv('./Data/epl_table_1314.csv')
table_14 = pd.read_csv('./Data/epl_table_1415.csv')
table_15 = pd.read_csv('./Data/epl_table_1516.csv')
table_16 = pd.read_csv('./Data/epl_table_1617.csv')
table_17 = pd.read_csv('./Data/epl_table_1718.csv')

table = [table_13, table_14, table_15, table_16, table_17]

In [36]:
# Season fixture
epl_detail_1314 = pd.read_csv('./Data/epl1314.csv').replace(team_name)[['HomeTeam', 'AwayTeam', 'FTR']]
epl_detail_1415 = pd.read_csv('./Data/epl1415.csv').replace(team_name)[['HomeTeam', 'AwayTeam', 'FTR']]
epl_detail_1516 = pd.read_csv('./Data/epl1516.csv').replace(team_name)[['HomeTeam', 'AwayTeam', 'FTR']]
epl_detail_1617 = pd.read_csv('./Data/epl1617.csv').replace(team_name)[['HomeTeam', 'AwayTeam', 'FTR']]
epl_detail_1718 = pd.read_csv('./Data/epl1718.csv').replace(team_name)[['HomeTeam', 'AwayTeam', 'FTR']]
epl_detail_1819 = pd.read_csv('./Data/epl1819.csv').replace(team_name)[['HomeTeam', 'AwayTeam', 'FTR']]

fixture = [epl_detail_1314, epl_detail_1415, epl_detail_1516, epl_detail_1617, epl_detail_1718]

In [37]:
epl_detail_1314.head(2)

,HomeTeam,AwayTeam,FTR
0,Arsenal,Aston Villa,A
1,Liverpool,Stoke City,H


In [38]:
table_13[['Team','H_Att', 'H_Def', 'A_Att', 'A_Def', 'HGF', 'HGA', 'AGA', 'AGF']].head(2)

,Team,H_Att,H_Def,A_Att,A_Def,HGF,HGA,AGA,AGF
0,Arsenal,1.204013,0.484581,1.409692,1.003344,36,11,30,32
1,Aston Villa,0.735786,1.277533,0.748899,1.070234,22,29,32,17


In [39]:
def result_percentage(dataframe, hometeam, awayteam):
    home_avg = dataframe['HGF'].sum()/380
    away_avg = dataframe['HGA'].sum()/380
    
    home_score = float(dataframe[dataframe['Team'] == hometeam]['H_Att']) * float(dataframe[dataframe['Team'] == awayteam]['A_Def']) * home_avg
    away_score = float(dataframe[dataframe['Team'] == awayteam]['A_Att']) * float(dataframe[dataframe['Team'] == hometeam]['H_Def']) * away_avg

    score = []
    # maximum score for a team is 5
    for goals in range(0, 6):
        scores = {}
        scores['Home'] = (poisson.pmf(goals, home_score) * 100) # Hometeam score
        scores['Away'] = (poisson.pmf(goals, away_score) * 100) # away score

        if len(scores) == 2:
                    score.append(scores)

    score = pd.DataFrame(score, columns=(['Home', 'Away']))

    # % of home team winning
    # home score > away score
    # home[1] * away[0]
    # home[2] * away[0] + home[2] * away[1]
    # home[3] * away[0] + home[3] * away[1] + home[3] * away[2]
    # home[4] * away[0] + home[4] * away[1] + home[4] * away[2] + home[4] * away[3]
    # home[5] * away[0] + home[5] * away[1] + home[5] * away[2] + home[5] * away[3]+  home[5] * away[4]
    home_w = 0
    away_w = 0
    draw = 0
    result = []
    for home in range(1, len(score)):
        for away in range(0, home):
            home_w += (score['Home'][home] * score['Away'][away] / 10000)
    result.append(home_w)

    for away in range(1, len(score)):
        for home in range(0, away):
            away_w += (score['Home'][home] * score['Away'][away] / 10000)
    result.append(away_w)

    for home in range(0, len(score)):
        away = home
        draw += (score['Home'][home] * score['Away'][away] / 10000)
    result.append(draw)

    return result

In [90]:
def decision(result_p):
    if result_p[2] >= 0.3:
        return 'D'
    elif abs(result_p[0] - result_p[1]) < 0.005:
        return 'D'
    elif result_p[2] > result_p[1] and result_p[2] > result_p[0]:
        return 'D'
    elif result_p[0] > result_p[1] and result_p[0] > result_p[2]:
        return 'H'
    elif result_p[1] > result_p[0] and result_p[1] > result_p[2]:
        return 'A'


In [91]:
def predict(fixture, rating):
    for i in range(len(fixture)):
        result = result_percentage(rating, fixture['HomeTeam'][i], fixture['AwayTeam'][i])
        fixture.loc[i, 'Predict'] = decision(result)

    fixture.loc[:, 'Result'] = (fixture.loc[:, 'FTR'] == fixture.loc[:, 'Predict']) * 1

In [92]:
for i in range(len(fixture)):
    predict(fixture[i], table[i])

In [93]:
print(epl_detail_1314['Result'].mean())
print(epl_detail_1415['Result'].mean())
print(epl_detail_1516['Result'].mean())
print(epl_detail_1617['Result'].mean())
print(epl_detail_1718['Result'].mean())
(epl_detail_1314['Result'].mean() + epl_detail_1415['Result'].mean() + 
 epl_detail_1516['Result'].mean() + epl_detail_1617['Result'].mean() + (epl_detail_1718['Result'].mean())) / 5


0.5736842105263158
0.5289473684210526
0.5157894736842106
0.6210526315789474
0.55


0.5578947368421053

In [95]:
epl_detail_1718

,HomeTeam,AwayTeam,FTR,Predict,Result
0,Arsenal,Leicester City,H,H,1
1,Brighton & Hove,Manchester City,A,A,1
2,Chelsea,Burnley,A,D,0
3,Crystal Palace,Huddersfield,A,H,0
4,Everton,Stoke City,H,H,1
5,Southampton,Swansea City,D,D,1
6,Watford,Liverpool,D,A,0
7,West Bromwich,Bournemouth,H,A,0
8,Manchester Utd,West Ham Utd,H,H,1
9,Newcastle Utd,Tottenham,A,A,1
